<a href="https://colab.research.google.com/github/luana-martins/Grafos/blob/main/grafos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seleção de Features para a Classificação de Dados de Saúde
## Jalisson Henrique, Luana Martins, Tiago Machado

Este projeto contém a implementação do algoritmo proposto para a disciplina de Grafos. Além disso, para possibilitar a execução do experimento, foram implementados quatro modelos de Machine Learning (Decision-Tree, KNN, Random-Forest e SVM-Linear). Ainda, contém a implementação do algoritmo Sequential Feature Selection (SFS), também utilizado para a comparação com o JLT.

**Importante:** 

*   *Configurar a Seção 6 conforme a execução que deseja!*
*   *Para executar vá no menu superior dessa página em "Ambiente de execução" e selecione "Executar tudo".*






# 1. Configuração do Ambiente

Nesta seção, as bibliotecas e o dataset são importados.


In [ ]:
#Importação das bibliotecas utilizadas
import pandas as pd
import numpy as np
 
#bibliotecas gráficas
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns;
 
#bibliotecas de métricas
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from warnings import filterwarnings
from sklearn.metrics import precision_recall_fscore_support
 
#Classificadores
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Algoritmos literatura
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# 2. Dataset

Nesta seção, o dataset é importado e descrito por meio de estatísticas descritivas.


In [ ]:
#Importação do dataset para variável 'dados'
dataset = pd.read_csv('https://raw.githubusercontent.com/luana-martins/Grafos/main/data.csv');

# Tratamento da label - M por (1), B por (0)
dataset['diagnosis'] = dataset['diagnosis'].replace('M',1)
dataset['diagnosis'] = dataset['diagnosis'].replace('B',0)

#Estatística básica do dataset
dataset.describe()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,0.627417,706.771388,19.289649,91.969033,654.889104,4.304801,4.835984,7.489124,2.366459,16.965766,0.851112,77.138555,825.490173,2549.980718,316.226116,0.007041,0.176469,1.153794,0.067979,0.231228,0.014329,315.194921,25.677223,107.261213,880.583128,10.633281,25.259112,26.723742,8.745685,30.367174,1.964313
std,1.250206e+08,0.483918,2430.243368,4.301036,24.298981,351.914129,21.074558,26.827478,35.618994,16.155145,53.846023,7.103493,277.327735,832.741506,1757.074266,1532.270716,0.003003,1.678798,17.470924,0.784389,2.112944,0.251388,1655.459336,6.146258,33.602542,569.356993,37.236433,96.473015,114.204035,39.465975,90.748044,14.464355
min,8.670000e+03,0.000000,7.760000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.116700,0.049960,0.111500,0.360200,0.771400,10.080000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,0.000000,12.210000,16.170000,75.170000,420.300000,0.086410,0.065260,0.029580,0.020310,0.163400,0.057800,0.236600,0.856100,1491.000000,18.520000,0.005169,0.013150,0.015090,0.007638,0.015180,0.002248,13.180000,21.080000,84.110000,515.300000,0.117800,0.150700,0.116800,0.064990,0.254900,0.071460
50%,9.060240e+05,1.000000,13.850000,18.840000,86.240000,551.100000,0.095940,0.094620,0.063870,0.033900,0.181400,0.061660,0.341600,1025.000000,2155.000000,25.790000,0.006380,0.020620,0.026020,0.010970,0.018780,0.003187,15.150000,25.410000,97.660000,686.500000,0.133800,0.227900,0.249200,0.101500,0.288400,0.080060
75%,8.813129e+06,1.000000,17.680000,21.800000,104.100000,782.700000,0.106100,0.132500,0.142500,0.077260,0.203600,0.066400,0.585800,1424.000000,3176.000000,49.850000,0.008146,0.032880,0.042560,0.014930,0.023700,0.004558,19.850000,29.720000,125.400000,1084.000000,0.150000,0.384200,0.431600,0.170800,0.331800,0.092110
max,9.113205e+08,1.000000,9904.000000,39.280000,188.500000,2501.000000,123.000000,277.000000,313.000000,162.000000,304.000000,78.000000,2873.000000,4885.000000,9807.000000,9833.000000,0.031130,27.000000,396.000000,12.000000,31.000000,6.000000,9981.000000,49.540000,251.200000,4254.000000,185.000000,1058.000000,1252.000000,291.000000,544.000000,173.000000


# 3. Classificadores

Nesta seção, são implementados os classificadores utilizados para o experimento, que são: (1) Classificador Linear, (2) Random-Forest, (3) KNN, e (4) Decision Tree. **Execute aqueles que deseja verificar**. 

In [ ]:
# (1) Classificador Linear 
# O que de fato queremos é uma função f([X1,X2,X3,Xn]) que nos retorne M (1) - Maligno ou B (0) - Benigno

# Recebe as entradas e o label para equação y = xi + e
def ClassificadorLinear(conjunto, imprimir):
  filterwarnings('ignore')

  y = dataset['diagnosis']
  x = []
  for variavel in conjunto:
    x.append(dataset[variavel])
  x = np.reshape(x, (len(conjunto), len(dataset))).T
  
  # Cria o modelo de machine learning (SVM)
  modelo = LinearSVC()

  # Divide o dataset em dados de treino e teste, com 70% treino e 30% teste
  treino_x, teste_x, treino_y, teste_y = train_test_split(x,y, train_size= .3, random_state = 0)

  # Treina o modelo criado
  modelo.fit(treino_x, treino_y)

  # Colhe o resultado do modelo ao dataset de teste 
  previsoes = modelo.predict(teste_x)
  acuracia = (accuracy_score(teste_y, previsoes) * 100)

  # Imprime os resultados de acurácia
  if imprimir:
    print("A acurácia do algoritmo de baseline foi %.2f%%" % acuracia)
    print("Quantidade de features selecionadas: ", len(conjunto))
    print("Lista de features selecioandas: ", conjunto)
    print('\n[Classification Report] SVM-kernel-linear')
    print( classification_report(teste_y, previsoes))

    # Matriz de confusão
    pd.DataFrame(confusion_matrix(teste_y, previsoes),
                index=['neg', 'pos'], columns=['pred_neg', 'pred_pos'])
  
  return acuracia

In [ ]:
# (2) Modelo RandomForest

# Recebe as entradas e o label para equação y = xi + e
def ModeloRandomForest(conjunto, imprimir):
  filterwarnings('ignore')

  y = dataset['diagnosis']
  x = []
  for variavel in conjunto:
    x.append(dataset[variavel])
  x = np.reshape(x, (len(conjunto), len(dataset))).T

  # Cria o modelo de machine learning (RandomForest)
  modelo = RandomForestClassifier(n_estimators=100)

  # Divide o dataset em dados de treino e teste, com 70% treino e 30% teste
  treino_x, teste_x, treino_y, teste_y = train_test_split(x,y, train_size= .3)

  # Treina o modelo criado
  modelo.fit(treino_x, treino_y)

  # Colhe o resultado do modelo ao dataset de teste 
  previsoes = modelo.predict(teste_x)
  acuracia = (accuracy_score(teste_y, previsoes) * 100)

  # Imprime os resultados de acurácia
  if imprimir:
    print("A acurácia do algoritmo de baseline foi %.2f%%" % acuracia)
    print("Quantidade de features selecionadas: ", len(conjunto))
    print("Lista de features selecioandas: ", conjunto) 
    print('\n[Classification Report] Random-Forest')
    print( classification_report(teste_y, previsoes))

    # Matriz de confusão
    pd.DataFrame(confusion_matrix(teste_y, previsoes),
                index=['neg', 'pos'], columns=['pred_neg', 'pred_pos'])
  
  return acuracia

In [ ]:
# (3) Classificador KNeighbors

#1. Recebe as entradas e o label para equação y = xi + e
def ModeloKNeighbors(conjunto, imprimir):
  filterwarnings('ignore')

  y = dataset['diagnosis']
  x = []
  for variavel in conjunto:
    x.append(dataset[variavel])
  x = np.reshape(x, (len(conjunto), len(dataset))).T
  
  # Cria o modelo de machine learning (knn)
  modelo = KNeighborsClassifier(n_neighbors=3) #n_neighbors=3 (número de vizinhos)

  # Divide o dataset em dados de treino e teste, com 70% treino e 30% teste
  treino_x, teste_x, treino_y, teste_y = train_test_split(x,y, train_size= .3)

  # Treina o modelo criado
  modelo.fit(treino_x, treino_y)

  # Colhe o resultado do modelo ao dataset de teste 
  previsoes = modelo.predict(teste_x)
  acuracia = (accuracy_score(teste_y, previsoes) * 100)

  # Imprime os resultados de acurácia
  if imprimir:
    print("A acurácia do algoritmo de baseline foi %.2f%%" % acuracia)
    print("Quantidade de features selecionadas: ", len(conjunto))
    print("Lista de features selecioandas: ", conjunto)
    print('\n[Classification Report] KNN')
    print( classification_report(teste_y, previsoes))

    # Matriz de confusão
    pd.DataFrame(confusion_matrix(teste_y, previsoes),
                index=['neg', 'pos'], columns=['pred_neg', 'pred_pos'])
  
  return acuracia

In [ ]:
# (4) Modelo DecisionTree

# Recebe as entradas e o label para equação y = xi + e
def ModeloDecisionTree(conjunto, imprimir):
  filterwarnings('ignore')

  y = dataset['diagnosis']
  x = []
  for variavel in conjunto:
    x.append(dataset[variavel])
  x = np.reshape(x, (len(conjunto), len(dataset))).T
  
  # Cria o modelo de machine learning (DecisionTree)
  modelo = DecisionTreeClassifier()

  # Divide o dataset em dados de treino e teste, com 70% treino e 30% teste
  treino_x, teste_x, treino_y, teste_y = train_test_split(x,y, train_size= .3)

  # Treina o modelo criado
  modelo.fit(treino_x, treino_y)

  # Colhe o resultado do modelo ao dataset de teste 
  previsoes = modelo.predict(teste_x)
  acuracia = (accuracy_score(teste_y, previsoes) * 100)

  # Imprime os resultados de acurácia
  if imprimir:
    print("A acurácia do algoritmo de baseline foi %.2f%%" % acuracia)
    print("Quantidade de features selecionadas: ", len(conjunto))
    print("Lista de features selecioandas: ", conjunto)    
    print('\n[Classification Report] Decision-Tree')
    print( classification_report(teste_y, previsoes))

    # Matriz de confusão
    pd.DataFrame(confusion_matrix(teste_y, previsoes),
                index=['neg', 'pos'], columns=['pred_neg', 'pred_pos'])
  
  return acuracia

# 4. Algoritmo de Seleção de feature - SFS

Nesta seção, é apresentada a implementação do algoritmo Sequential Feature Selection. 

In [ ]:
# (1) Sequential Feature Selection

def SequentialFeatureSelection(conjunto, classificador):
  filterwarnings('ignore')

  y = dataset['diagnosis']
  x = []
  for variavel in conjunto:
    x.append(dataset[variavel])
  x = np.reshape(x, (len(conjunto), len(dataset))).T

  # Divide o dataset em dados de treino e teste, com 70% treino e 30% teste
  treino_x, teste_x, treino_y, teste_y = train_test_split(x,y, train_size= .3)
  treino_x.shape, teste_x.shape

  feature_names = conjunto  
  # chama o algoritmo para seleção de features e treina o modelo
  if classificador == "classificadorLinear":
    sfs = SFS(LinearSVC(random_state=0),
              k_features = (1, len(conjunto)),
              forward= True,
              floating = False,
              verbose= 2,
              scoring= 'accuracy',
              cv = 4,
              n_jobs= -1
              )
    sfs.fit(treino_x, treino_y,  custom_feature_names=feature_names)
    features = np.array(sfs.k_feature_names_)
    ClassificadorLinear(features, True)

  if classificador == "classificadorRandomForest":
    sfs = SFS(RandomForestClassifier(n_estimators=100),
              k_features = (1, len(conjunto)),
              forward= True,
              floating = False,
              verbose= 2,
              scoring= 'accuracy',
              cv = 4,
              n_jobs= -1
              )
    sfs.fit(treino_x, treino_y,  custom_feature_names=feature_names)
    features = np.array(sfs.k_feature_names_)
    ModeloRandomForest(features, True)

  if classificador == "classificadorDecisionTree":
    sfs = SFS(DecisionTreeClassifier(),
              k_features = (1, len(conjunto)),
              forward= True,
              floating = False,
              verbose= 2,
              scoring= 'accuracy',
              cv = 4,
              n_jobs= -1
              )
    sfs.fit(treino_x, treino_y,  custom_feature_names=feature_names)
    features = np.array(sfs.k_feature_names_)
    ModeloDecisionTree(features, True)

  if classificador == "classificadorKNN": 
    sfs = SFS(KNeighborsClassifier(n_neighbors=3),
              k_features = (1, len(conjunto)),
              forward= True,
              floating = False,
              verbose= 2,
              scoring= 'accuracy',
              cv = 4,
              n_jobs= -1
              )
    sfs.fit(treino_x, treino_y,  custom_feature_names=feature_names)
    features = np.array(sfs.k_feature_names_)
    ModeloKNeighbors(features, True)


# 5. O Algoritmo Proposto
Nesta seção, apresentamos nossa proposta, o algoritmo **JLT**. Ele consiste na aplicação do método filter e do método wrapper para a seleção de um conjunto de features. Eles são combinados em duas etapas. Na primeira etapa, **Tupla Solução**, o método filter calcula a correlação entre label-features para selecionar uma dupla de features <forte, fraco>. O resultado desse filtro serve como entrada para o método wrapper que classifica cada dupla <forte, fraco> para escolher uma promissora. Na segunda etapa, **Conjunto Solução**, o método filtro é aplicado para selecionar um conjunto de features complementares à Tupla Solução. Os resultados desse filtro são utilizados como entrada no método wrapper, que classifica as features complementares para selecionar o conjunto final necessário para treinar o modelo. **Para execução do algoritmo JLT, execute esta seção e configure a Seção 6!**

In [ ]:
def JLT():

  ######### TuplaSolução: seleciona uma tupla<forte, fraco> como solução inicial
  correlacaoForteLabel = []
  correlacaoFracaLabel = []
  
  #### Filter pt 1
  # Cria listas de correlações, entre label e cada coluna do dataset
  for coluna in dataset:

    # As correlações entre label-label e label-id não são necessárias
    if (coluna != 'diagnosis') and (coluna != 'id'):

      # O teste de Spearman é usado para distribuições normais e não normais
      coef, p_value = stats.spearmanr(dataset['diagnosis'], dataset[coluna])
      coef = abs(coef)

      # Insere a correlação na lista correspondente
      if (coef >= 0 and coef <= 0.4):
        correlacaoFracaLabel.append([coluna, coef])
      if (coef >= 0.6 and coef <= 1):
        correlacaoForteLabel.append([coluna, coef])

  # Ordena das correlações para facilitar a montagem de tuplas<forte, fraco>
  correlacaoForteLabel.sort(key=lambda tup: tup[1], reverse=True)
  correlacaoFracaLabel.sort(key=lambda tup: tup[1])

  #### Wrapper pt 1
  featuresJaAnalisadas = []
  solucaoOtima = 0.0
  conjuntoSolucao = []

  # Percorre as features fortes em ordem decrescente
  for forte in correlacaoForteLabel:

    # Se a feature já foi analisada como forte/fraca, não deve ser analisada novamente
    if forte[0] not in featuresJaAnalisadas:
      featuresJaAnalisadas.append(forte[0])

    # Percorre as features fracas em ordem crescente
    for fraco in correlacaoFracaLabel:
      
      # Se a feature já foi analisada como forte/fraca, não deve ser analisada novamente
      if fraco[0] not in featuresJaAnalisadas:
        featuresJaAnalisadas.append(fraco[0])
      
      # Cria duplas forte-fraco
      solucaoInicial = []
      solucaoInicial.append([forte[0], fraco[0]])
      ultimaPosicao = len(solucaoInicial)-1

      # Calcula a acurácia e, enquanto o novo conjunto melhorar o valor da acurácia, o conjuntoSolucao é atualizado
      acuracia = Classificador(solucaoInicial[ultimaPosicao], False)
      if acuracia > solucaoOtima:
        solucaoOtima = acuracia
        conjuntoSolucao.append([solucaoInicial, solucaoOtima])
      else:
        break
  

  ######### ConjuntoSolução: seleciona o conjunto final de features

  ### Filter pt 2 
  ultimaPosicao = len(conjuntoSolucao)-1
  tupla = conjuntoSolucao[ultimaPosicao]
  solucaoNova = [tupla[0][0][0], tupla[0][0][1]]
  featuresCandidatas = []
  
  feature_forte_normalidade = verificaDistribuicao(dataset[solucaoNova[0]]) 
  feature_fraca_normalidade = verificaDistribuicao(dataset[solucaoNova[0]])

  # Percorre features ainda não analisadas 
  for feature in dataset:
    propriedade_nao_linearidade = True 
    propriedade_nao_negatividade = True
    coef_feature_forte = 0
    coef_feature_fraca = 0

    if (feature not in featuresJaAnalisadas) and (feature != 'diagnosis') and (feature != 'id'): 

      # Dados não-normais satisfazem não linearidade, porque não é possivel aplicar Pearson
      if (not feature_forte_normalidade) or (not feature_fraca_normalidade):
        coef_feature_forte, p_value = stats.spearmanr(dataset[solucaoNova[0]], dataset[feature])
        coef_feature_fraca, p_value = stats.spearmanr(dataset[solucaoNova[1]], dataset[feature])

      # Dados normais devem ter correlação igual a 0 para satisfazer linearidade
      if (feature_forte_normalidade) and (feature_fraca_normalidade):
        feature_complementar = verificaDistribuicao(dataset[feature])
        coef_feature_forte, coef_feature_fraca

        # Feature complementar também deve ter distribuição normal
        if (feature_complementar):
          coef_feature_forte, p_value = stats.pearsonr(dataset[solucaoNova[0]], dataset[feature])
          coef_feature_fraca, p_value = stats.pearsonr(dataset[solucaoNova[1]], dataset[feature])
          
          # Se for diferente de 0 é porque há uma correlação linear 
          if (coef_feature_forte != 0) and (coef_feature_fraca != 0):
            propriedade_nao_linear = False          
      
      # Propriedade de não-negatividade para distribuição não-normal
      if (coef_feature_forte < 0) and (coef_feature_fraca < 0):
          propriedade_nao_negatividade = False

      # Se as propriedades de não-linearidade e não-negatividade são satisfeitas, tentar adicionar a feture no conjunto
      if (propriedade_nao_negatividade and propriedade_nao_linearidade):
        featuresCandidatas.append([feature, max(coef_feature_forte, coef_feature_fraca)])


  featuresCandidatas.sort(key=lambda tup: tup[1])
  ### Wrapper pt 2
  for feature in featuresCandidatas:
    solucaoNova.append(feature[0])
    acuracia = Classificador(solucaoNova, False)
    if acuracia > solucaoOtima:
      solucaoOtima = acuracia
      conjuntoSolucao.append([solucaoNova, solucaoOtima])
    else:
      solucaoNova.pop()

  Classificador(solucaoNova, True)


# Função para verificar distribuicao dos dados
def verificaDistribuicao(amostra):
  media = np.mean(amostra)
  std = np.std(amostra)

  # A normalidade é verificada por meio do teste de Kolmogorov-Smirnov
  ks_stat, ks_p_valor = stats.kstest(amostra, cdf='norm', args=(media, std), N = len(dataset))   
  
  # Se o p-valor as ks_stat for menor ou igual que 0.05, então a distribuição não é normal
  if ks_stat <= 0.05:
    return 0
  return 1     

#6. Configuração
Nesta seção, a execução do projeto deve ser configurada. Primeiramente, deve ser realizada a **(1) Seleção do Classificador**, onde um dos classificadores devem ser marcados como True: SVM-Linear, Random-Forest, KNN ou Decision-Tree. Em seguida, deve ser configurado o modo como o classificador será executado em **(2) Seleção de Features**, onde pode uma das opções deve ser marcada como True: execução do algoritmo proposto JLT, execução sem seleção de features, execução do SFS.

**Marque apenas uma opção como TRUE no bloco (1) e outra no bloco (2)!**
 

In [ ]:
# (1) Seleção do classificador a ser executado, selecione 1 por vez! 
classificadorLinear = False
classificadorDecisionTree = False
classificadorRandomForest = False
classificadorKNN = True

# (2) Seleção de como rodar o classificador, selecione 1 por vez!
semSelecaoDeFeatures = False
selecaoDeFeaturesJLT = True
selecaoDeFeaturesSFS = False


# Função para executar a configuração selecionada
def Classificador(conjunto, imprimir):

  if classificadorLinear:
    return ClassificadorLinear(conjunto, imprimir)
  
  if classificadorRandomForest:
    return ModeloRandomForest(conjunto, imprimir)
  
  if classificadorKNN:
    return ModeloKNeighbors(conjunto, imprimir)
  
  if classificadorDecisionTree: 
    return ModeloDecisionTree(conjunto, imprimir)

# Função para executar o modo selecionado
def ModoExecucao():
  lista = []
  for feature in dataset:
    if (feature != 'diagnosis') and (feature != 'id'):
      lista.append(feature)
  
  if semSelecaoDeFeatures:
    Classificador(lista, True)
  
  if selecaoDeFeaturesJLT:
    JLT()

  if selecaoDeFeaturesSFS:
    if classificadorLinear:
      SequentialFeatureSelection(lista, "classificadorLinear")
    if classificadorRandomForest:
      SequentialFeatureSelection(lista, "classificadorRandomForest")
    if classificadorKNN:
      SequentialFeatureSelection(lista, "classificadorKNN")
    if classificadorDecisionTree:
      SequentialFeatureSelection(lista, "classificadorDecisionTree")


ModoExecucao()